<img src="./img/oscon.png" width="400">


In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data

import sys

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [3]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [4]:
def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)


def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b


In [6]:
# The image input -- a 1X784 vector 
x = tf.placeholder(tf.float32, shape=[None, 784])

# The correct label
y_ = tf.placeholder(tf.float32, shape=[None, 10])


# With CNNs we have a spatial notion, and need the image in the correct shape!
x_image = tf.reshape(x, [-1, 28, 28, 1])

# First conv layer
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

# Second conv layer
conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

# We flatten the rectangular image representation before the fully-connected part 
conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

# Dropout
keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

# Readout layer 
y_conv = full_layer(full1_drop, 10)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
DATA_DIR = '/tmp/data' if not 'win' in sys.platform else "c:\\tmp\\data"
STEPS = 2000
MINIBATCH_SIZE = 50
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting c:\tmp\data\train-images-idx3-ubyte.gz
Extracting c:\tmp\data\train-labels-idx1-ubyte.gz
Extracting c:\tmp\data\t10k-images-idx3-ubyte.gz
Extracting c:\tmp\data\t10k-labels-idx1-ubyte.gz


In [9]:
sess =  tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(STEPS):
    batch = mnist.train.next_batch(MINIBATCH_SIZE)

    if i % 200 == 0:
        valid_accuracy = sess.run(accuracy, 
                                  feed_dict={x: mnist.validation.images, 
                                             y_: mnist.validation.labels,
                                             keep_prob: 1.0})
        print("step {}, validation accuracy {}".format(i, valid_accuracy))

    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})



step 0, validation accuracy 0.1281999945640564
step 200, validation accuracy 0.906599760055542
step 400, validation accuracy 0.9367997646331787
step 600, validation accuracy 0.956599771976471
step 800, validation accuracy 0.9605996608734131
step 1000, validation accuracy 0.9673997163772583
step 1200, validation accuracy 0.9691997766494751
step 1400, validation accuracy 0.9741997122764587
step 1600, validation accuracy 0.9725996851921082
step 1800, validation accuracy 0.9733997583389282


In [10]:
# Split the test part into 10 equal segments 
X = mnist.test.images.reshape(10, 1000, 784)
Y = mnist.test.labels.reshape(10, 1000, 10)
test_accuracy = np.mean([sess.run(accuracy, feed_dict={x:X[i], y_:Y[i], 
                                                       keep_prob:1.0}) for i in range(10)])

print("test accuracy: {:.4f}%".format(test_accuracy*100.))

test accuracy: 97.5600%


In [11]:
sess.close()